# Mistral Fine-tuning API

Check out the docs: https://docs.mistral.ai/capabilities/finetuning/

In [ ]:
#!pip install mistralai pandas

## Prepare the dataset

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required jsonl format for fine-tuning.

In [2]:
import pandas as pd
df = pd.read_json('./video_script/data/generated_video_conversation.jsonl', lines=True)


df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("videos_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("videos_chunk_eval.jsonl", orient="records", lines=True)

In [3]:
!ls -lh

total 306072
-rw-r--r--   1 pierrebittner  staff   6.5K Jun 19 16:19 Untitled.ipynb
drwxr-xr-x   5 pierrebittner  staff   160B Jun 28 16:31 __pycache__
-rw-r--r--   1 pierrebittner  staff   1.1K Jun 21 15:53 concatenate.py
drwxr-xr-x   4 pierrebittner  staff   128B Jun 21 18:10 data
-rw-r--r--@  1 pierrebittner  staff    39K Jun 21 16:19 mistral_finetune_api.ipynb
-rw-r--r--   1 pierrebittner  staff    57K Jun 21 18:37 mistral_finetune_api_news.ipynb
-rw-r--r--   1 pierrebittner  staff    61K Jun 28 17:44 mistral_finetune_api_video.ipynb
drwxr-xr-x   5 pierrebittner  staff   160B Jun 27 22:41 news
-rw-r--r--   1 pierrebittner  staff    20K Jun 21 18:11 news_chunk_eval.jsonl
-rw-r--r--   1 pierrebittner  staff   3.5M Jun 21 18:11 news_chunk_train.jsonl
-rw-r--r--   1 pierrebittner  staff   447B Jun 21 11:29 prompts.py
-rw-r--r--   1 pierrebittner  staff   3.3K Jun 19 11:53 reformat_data.py
drwxr-xr-x   4 pierrebittner  staff   128B Jun 21 13:28 tests
-rw-r--r--   1 pierrebittner  staff 

## Reformat dataset
If you upload this ultrachat_chunk_train.jsonl to Mistral API, you might encounter an error message “Invalid file format” due to data formatting issues. To reformat the data into the correct format, you can download the reformat_dataset.py script and use it to validate and reformat both the training and evaluation data:

In [ ]:
# download the validation and reformat script
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

In [4]:
# validate and reformat the training data
!python reformat_data.py videos_chunk_train.jsonl

In [5]:
# validate the reformat the eval data
!python reformat_data.py videos_chunk_eval.jsonl

In [6]:
df_train.iloc[104]['messages']

[{'role': 'user',
  'content': "As a 'Youtube Video Script Writer' for 'GenAI and LLM powered application' influencer,\nyour task is to refine video script to ensure they are engaging and meet the expected style and structure.\n\nRead the transcript carefully and point out all stylistic issues according\nto the following writing style guide. \n\nThe video must follow those writing tips:\n 1) Use Short Sentences.\n 2) Use the Present Tense.\n 3) Use first person\n 3) Write in a Conversational Style.\n 4) Use More Active Voice Than Passive Voice.\n 5) Be Clear and simple: translates jargon into simpler words.\n 6) Sprinkle in Some Humor.\n 7) Avoid repetition.\n 8) Avoid conventional messages and overdoing it with words like “cutting edge”, “revolutionize\n 9) Be Confident: removes words that undermine authority.\n 10) Be Concise: makes writing more digestible with fewer than 25 words in a sentence,\n fewer than 4 sentences per paragraph, and no double descriptions.\n\nThe video must fol

## Upload dataset

In [17]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("videos_chunk_train.jsonl", "rb") as f:
    videos_chunk_train = client.files.create(file=("videos_chunk_train.jsonl", f))
with open("videos_chunk_eval.jsonl", "rb") as f:
    videos_chunk_eval = client.files.create(file=("videos_chunk_eval.jsonl", f))

In [18]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [19]:
pprint(videos_chunk_train)

{
    "id": "048ac6dd-6636-467f-9326-c46c15d64e0a",
    "object": "file",
    "bytes": 4691026,
    "created_at": 1719608328,
    "filename": "videos_chunk_train.jsonl",
    "purpose": "fine-tune"
}


In [20]:
pprint(videos_chunk_eval)

{
    "id": "1d871376-0334-4bd5-b431-746f65772254",
    "object": "file",
    "bytes": 24110,
    "created_at": 1719608328,
    "filename": "videos_chunk_eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [21]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="mistral-small-latest",#"mistral-small-latest", #"open-mistral-7b", 
    training_files=[videos_chunk_train.id],
    validation_files=[videos_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)

In [22]:
pprint(created_jobs)

{
    "id": "f732bd21-cb32-40bd-bdb7-0546e6eed0c6",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "mistral-small-latest",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719608352,
    "modified_at": 1719608352,
    "training_files": [
        "048ac6dd-6636-467f-9326-c46c15d64e0a"
    ],
    "validation_files": [
        "1d871376-0334-4bd5-b431-746f65772254"
    ],
    "object": "job",
    "integrations": []
}


In [23]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "f732bd21-cb32-40bd-bdb7-0546e6eed0c6",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "mistral-small-latest",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719608352,
    "modified_at": 1719608352,
    "training_files": [
        "048ac6dd-6636-467f-9326-c46c15d64e0a"
    ],
    "validation_files": [
        "1d871376-0334-4bd5-b431-746f65772254"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719608352
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "created_at": 1719608352
        }
    ],
    "checkpoints": [],
    "estimated_start_time": null
}
Job is RUNNING, waiting 10 seconds
{
    "id": "f732bd21-cb

In [24]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "f732bd21-cb32-40bd-bdb7-0546e6eed0c6",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:mistral-small-latest:c056c2e4:20240628:f732bd21",
            "model": "mistral-small-latest",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719608352,
            "modified_at": 1719608545,
            "training_files": [
                "048ac6dd-6636-467f-9326-c46c15d64e0a"
            ],
            "validation_files": [
                "1d871376-0334-4bd5-b431-746f65772254"
            ],
            "object": "job",
            "integrations": []
        },
        {
            "id": "8e2d2916-7a00-470d-b990-aa53a2fadaaf",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral

In [25]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "f732bd21-cb32-40bd-bdb7-0546e6eed0c6",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:mistral-small-latest:c056c2e4:20240628:f732bd21",
    "model": "mistral-small-latest",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719608352,
    "modified_at": 1719608545,
    "training_files": [
        "048ac6dd-6636-467f-9326-c46c15d64e0a"
    ],
    "validation_files": [
        "1d871376-0334-4bd5-b431-746f65772254"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1719608545
        },
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719608352
        },
        {
            "name": "status-updated",
            "data": {
    

## Use a fine-tuned model

In [26]:
role = "As a 'Youtube Video Script Writer' for 'GenAI and LLM powered application' influencer"

content = (f"{role}, "
    f"your task is to write the script for an engaging video of 5 to 10 minutes (1000 to 2000 words)."
    f"The script should include a title, the transcript, the author name and a publication date."
    f"The video topic is 'Generative AI with LLMs'"
    f"Here is a  short description: 'Understand the generative AI lifecycle. Describe transformer architecture powering LLMs. Apply training/tuning/inference methods. Hear from researchers on generative AI challenges/opportunities.'"
    f"Skill level of audience is 'Intermediate', the presenter will be 'Mike Chambers'.\n\n")


In [27]:
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role="user", content=content)]
)

In [28]:
pprint(chat_response)

{
    "id": "460303ed5b8a489593ea398f93d4b06c",
    "object": "chat.completion",
    "created": 1719608600,
    "model": "ft:mistral-small-latest:c056c2e4:20240628:f732bd21",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Title: \"Generative AI with LLMs: Unleashing the Power of Transformers\"\n\nAuthor: Mike Chambers\n\nPublication Date: 20th February 2023\n\n---\n\nTranscript:\n\nHi, fellow AI enthusiasts! Welcome back to our channel. I'm Mike Chambers and today we're diving into the fascinating world of Generative AI with LLMs, or Large Language Models. If you're new here, don't forget to hit that subscribe button and ring the bell so you never miss our updates.\n\n[00:00 - 00:30] Introduction\n\"Generative AI with LLMs: Unleashing the Power of Transformers. In this video, we'll explore the lifecycle of generative AI, delve into the transformer architecture powering LLMs, and discuss traini

In [29]:
response_ori = client.chat(
        model="mistral-small-latest",
        messages=[ChatMessage(role="user", content=content)],
    )

In [30]:
pprint(response_ori)

{
    "id": "ddb38f49de494d978fd652b20972c600",
    "object": "chat.completion",
    "created": 1719608639,
    "model": "mistral-small-latest",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Title: Unlocking the Power of Generative AI with LLMs: A Deep Dive\n\nTranscript:\n\n[INTRO]\n\nMike Chambers: \"Hello, everyone! Welcome back to our channel. Today, we're diving deep into the fascinating world of Generative AI with Language Model Learning, or LLMs. If you're new here, don't forget to hit that subscribe button and the bell icon so you won't miss our future videos.\n\n[SEGMENT 1: UNDERSTANDING GENERATIVE AI]\n\nMike: Generative AI is a subset of machine learning that focuses on creating new content. It's like teaching a computer to paint a picture or write a poem. But how does it work?\n\n[SEGMENT 2: TRANSFORMER ARCHITECTURE]\n\nMike: The secret sauce behind many LLMs is the Transformer ar

In [31]:
news_1 = response_ori.choices[0].message.content

In [32]:
news_2 = chat_response.choices[0].message.content

In [33]:
evaluation_framework = """# Evaluation Framework
Point attribution should be lowered. In case of 0.5 point, give 0.

## Evaluation of the Tone - 12 points

The video must follow these writing styles:
1. Conciseness: use Short Sentences (less than 25 words per sentence): 1 point
2. Use the Present Tense: 1 point
3. Use first person: 1 point
4. Write in a conversational style: 1 point
5. Use more active voice than passive voice: 1 point
6. Keep it simple - no jargon employed: 1 point
7. Sprinkle in some Humor: 1 point
8. Avoid repetition: 1 point
9. Avoid conventional messages: 1 point
10. Avoid overdoing it or over-sensational with words like “cutting edge”, “revolutionize”: 1 point
11. Confident: no words that undermine authority: 1 point
12. Energetic and Enthusiastic Tone: 1 point

## Evaluation of the Structure and Content: 12 points

### Section 1: Video hook and intro: 6 points
- Does the script provide enough context for the video to make sense? 1 point
- Does the Stakes and payoff are introduce to know why we should watch until the end? 1 point
- A curiosity gap is created: What viewers want to know and not all information is given away. 1 point
- Leverage input bias: the effort (time, energy, money) that went into the video is showed. 1 point
- The video body starts no later than the 20-second mark 1 point
- Includes an engaging story or comparison to make the topic relatable 1 point
### Section 2: Body, main content, and research: 4 points
- Consistent contrast is incorporated to keep things from getting stale 1 point
- Good pacing: cycles of high energy and low energy are alternated 1 point
    - Each cycle should be 2 - 4 minutes long, with shorter cycles in the beginning and longer cycles at the end 1 point
    - The last 20% of long video can be used for slower content, while the beginning of the video is allocated for lighter, faster content 1 point
- Critical analysis and personal insights are included 1 point
- Practical, real-world applications of the technologies are discussed 1 point
- Balanced optimism and realism 1 point
### Section 3: CTA (call to action) and conclusion: 2 points
- Conclusion leaves a lasting impression by revealing the payoff 1 point
- Ends on a high note, either dramatic, wholesome, or funny 1 point

## Global score (10): half tone, half structure, and content
"""

video_structure = """The video must follow this structure :\n 
- [Video hook and introduction]\n
- [Body content]\n
- [Conclusion and call to action]\n"""

writing_tips = """The video must follow those writing tips:
 1) Use Short Sentences.
 2) Use the Present Tense.
 3) Use first person
 3) Write in a Conversational Style.
 4) Use More Active Voice Than Passive Voice.
 5) Be Clear and simple: translates jargon into simpler words.
 6) Sprinkle in Some Humor.
 7) Avoid repetition.
 8) Avoid conventional messages and overdoing it with words like “cutting edge”, “revolutionize
 9) Be Confident: removes words that undermine authority.
 10) Be Concise: makes writing more digestible with fewer than 25 words in a sentence,
 fewer than 4 sentences per paragraph, and no double descriptions."""

In [34]:
response_comparison = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""{role},
        your task is to refine and rewrite videos transcript to ensure they meet the expected video structure and writing tips.
        {video_structure}
        {writing_tips}
        You are now given two video transcripts.
        Read the script carefully and point which one has the most stylistic issues according
        to the style guide. Do not rewrite the news articles.
        <News_1>{news_1}</News_1>
                      
        <News_2> {news_2}</News_2>""")]
    )

In [35]:
response_comparison.choices[0].message.content

'Both scripts generally adhere to the required video structure and writing tips. However, News_2 has more stylistic issues compared to News_1.\n\n1) News_2 uses longer sentences, which can make the content harder to digest. For example, the sentence "Transformers are the backbone of LLMs like BERT and GPT-3. They process sequences of data in parallel, enabling them to capture long-range dependencies and understand context better." could be split into two sentences to improve readability.\n\n2) News_2 uses passive voice in some instances, such as "data is fed into the model" and "model is adjusted." News_1, on the other hand, consistently uses active voice.\n\n3) News_2\'s conclusion could be more concise. It repeats some information from the introduction and could be shortened to maintain viewer engagement.\n\n4) News_2 uses more jargon without immediately explaining it, such as "BERT" and "GPT-3." While these terms may be familiar to some viewers, they could be confusing to others. Ne

In [36]:
response_critique = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""{role},
        your task is to refine and rewrite videos transcript to ensure they meet the high standards of clarity,
        precision, and sophistication characteristic of the influencer.
        You are now given a evaluation framework.
        {evaluation_framework}
        Read the transcript carefully and point out all stylistic issues of the given script according
        to the framework. Do not rewrite the script. 
        Finaly grade the script from 1 to 10 based on the level of compliance of the evaluation framework.
        
        <News>{news_2}</News>
        
        Critique:
        
        Grade:""")]
    )

In [37]:
response_critique.choices[0].message.content

'Critique:\n\nTone:\n\n1. Conciseness: The script is generally concise, but some sentences could be shorter, especially in the sections "Generative AI Lifecycle" and "Training, Tuning, and Inference".\n2. Use of Present Tense: The script consistently uses the present tense.\n3. Use of First Person: The script uses the first person effectively.\n4. Conversational Style: The script is mostly conversational, but some sections sound more academic than conversational.\n5. Active Voice: The script uses more active voice than passive voice.\n6. Simplicity: The script avoids jargon but could simplify some technical terms for a broader audience.\n7. Humor: The script lacks humor.\n8. Repetition: The script avoids repetition.\n9. Conventional Messages: The script avoids conventional messages.\n10. Over-sensational: The script avoids over-sensational language.\n11. Confidence: The script is confident and authoritative.\n12. Energetic and Enthusiastic Tone: The script is generally energetic and en

In [38]:
response_critique = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""{role},
        your task is to refine and rewrite videos transcript to ensure they meet the high standards of clarity,
        precision, and sophistication characteristic of the influencer.
        You are now given a evaluation framework.
        {evaluation_framework}
        Read the transcript carefully and point out all stylistic issues of the given script according
        to the framework. Do not rewrite the script. 
        Finaly grade the script from 1 to 10 based on the level of compliance of the evaluation framework.
        
        <News>{news_1}</News>
        
        Critique:
        
        Grade:""")]
    )

In [39]:
response_critique.choices[0].message.content

'Critique:\n\n1. Conciseness: The script is generally concise, but some sentences could be shortened to meet the 25-word limit. For instance, "Generative AI is a subset of machine learning that focuses on creating new content. It\'s like teaching a computer to paint a picture or write a poem. But how does it work?" could be shortened to "Generative AI creates new content, like teaching a computer to paint or write. How does it work?"\n\n2. Use of Present Tense: The script consistently uses the present tense.\n\n3. Use of First Person: The script uses the first person effectively.\n\n4. Conversational Style: The script is generally conversational, but could be improved by adding more direct audience engagement.\n\n5. Active Voice: The script uses active voice effectively.\n\n6. Simplicity: The script is generally simple, but some technical terms like "Transformer architecture" and "LLMs" may be considered jargon to a non-technical audience.\n\n7. Humor: The script lacks humor.\n\n8. Rep

## Integration with Weights and Biases
We can also offer support for integration with Weights & Biases (W&B) to monitor and track various metrics and statistics associated with our fine-tuning jobs. To enable integration with W&B, you will need to create an account with W&B and add your W&B information in the “integrations” section in the job creation request:



In [ ]:
from mistralai.models.jobs import WandbIntegrationIn

WANDB_API_KEY = "XXX"

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ultrachat_chunk_train.id],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=100,
        learning_rate=0.0001,
    ),
    integrations=[
        WandbIntegrationIn(
            project="test_ft_api",
            run_name="test",
            api_key=WANDB_API_KEY,
        ).dict()
    ],
)